In [2]:
import pandas as pd
df = pd.read_csv("../data/dedup_data.csv")
df.info()
df.head()
df[['given_name','surname','postcode']].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   given_name     4844 non-null   object 
 1   surname        4921 non-null   object 
 2   street_number  4755 non-null   float64
 3   address_1      4846 non-null   object 
 4   address_2      4307 non-null   object 
 5   suburb         4915 non-null   object 
 6   postcode       5000 non-null   int64  
 7   state          4915 non-null   object 
 8   date_of_birth  4845 non-null   float64
 9   soc_sec_id     5000 non-null   int64  
 10  id             5000 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 429.8+ KB


,postcode
count,5000.000000
mean,3689.369400
std,1418.253235
min,331.000000
25%,2500.000000
50%,3197.000000
75%,4670.000000
max,9396.000000


In [3]:
print(df.isnull().sum())

print("Sample date_of_birth values:")
print(df[df['date_of_birth'].notnull()]['date_of_birth'].head())

print("\nPostcode length distribution:")
print(df['postcode'].astype(str).str.len().value_counts())

print("\nString column summary:")
print(df.describe(include='object'))

# Specifically check 'state' as it's a good blocking candidate
print("\nState values:")
print(df['state'].value_counts())

given_name       156
surname           79
street_number    245
address_1        154
address_2        693
suburb            85
postcode           0
state             85
date_of_birth    155
soc_sec_id         0
id                 0
dtype: int64
Sample date_of_birth values:
0    19560409.0
1    19080419.0
2    19081128.0
3    19921119.0
4    19991207.0
Name: date_of_birth, dtype: float64

Postcode length distribution:
postcode
4    4964
3      36
Name: count, dtype: int64

String column summary:
       given_name surname              address_1       address_2     suburb  \
count        4844    4921                   4846            4307       4915   
unique       1213    1740                   2358            2303       1706   
top        joshua   white  newman morris circuit  brentwood vlge  frankston   
freq           81     123                     18              32         44   

       state  
count   4915  
unique    35  
top      nsw  
freq    1581  

State values:
state
nsw    15

In [4]:
print(f"\nTotal unique soc_sec_id: {df['soc_sec_id'].nunique()}")
print("Most common soc_sec_ids (Top 10):")
print(df['soc_sec_id'].value_counts().head(10))

print(f"\nTotal rows: {len(df)}")


Total unique soc_sec_id: 2291
Most common soc_sec_ids (Top 10):
soc_sec_id
9305219    6
4669127    6
2661643    6
7462490    6
1677968    6
5449129    6
9687938    6
8928429    6
4282147    6
2510937    6
Name: count, dtype: int64

Total rows: 5000


# Pre processing

In [5]:
import pandas as pd

# Load the original dataframe again (just to be safe)
df = pd.read_csv("../data/dedup_data.csv")
df_clean = df.copy()

# 1. Fix Data Types and Standardize

# POSTCODE: Convert to string, pad with leading zeros to 4 digits
# (Handles the '331' -> '0331' problem)
df_clean['postcode'] = df_clean['postcode'].astype(str).str.zfill(4)

# STREET NUMBER: Convert to string, fill NaNs, remove '.0'
df_clean['street_number'] = df_clean['street_number'].fillna('').astype(str).str.replace(r'\.0$', '', regex=True)

# DATE_OF_BIRTH: Convert YYYYMMDD.0 float to 'YYYY-MM-DD' string
# Use pd.to_datetime for robust conversion, handling errors
df_clean['date_of_birth'] = pd.to_datetime(
    df_clean['date_of_birth'].fillna(0).astype(int).astype(str),
    format='%Y%m%d',
    errors='coerce' # If any date is invalid (e.g., 0), make it NaT (Not a Time)
)
# Now convert the valid dates to a standard string format
df_clean['date_of_birth'] = df_clean['date_of_birth'].dt.strftime('%Y-%m-%d').fillna('')


# 2. Clean and Normalize String Columns

# Fill NaNs in all object columns with an empty string
str_cols = df_clean.select_dtypes(include='object').columns
df_clean[str_cols] = df_clean[str_cols].fillna('')

# Normalize all string columns: lowercase and strip whitespace
for col in str_cols:
    if col != 'postcode': # We already processed postcode
        df_clean[col] = df_clean[col].str.lower().str.strip()

# STATE: Manually fix the typos we found in the EDA
state_map = {
    'nws': 'nsw', 'nsq': 'nsw', 'nxw': 'nsw', 'nss': 'nsw', 
    'nsh': 'nsw', 'nhw': 'nsw', 'nsy': 'nsw', 'nse': 'nsw',
    'vci': 'vic', 'vid': 'vic', 'vix': 'vic', 'viv': 'vic', 'vkc': 'vic',
    'qdl': 'qld', 'qls': 'qld', 'qlf': 'qld', 'qle': 'qld', 'qkd': 'qld', 'wq': 'qld',
    'aw': 'wa', 'ws': 'wa',
    'as': 'sa', 'ss': 'sa', 'sic': 'sa',
    'tsa': 'tas',
    'sct': 'act',
    'nf': 'nt' # Assuming nf is northern territory? Or maybe norfolk island. Safe to map to nt for now.
}
df_clean['state'] = df_clean['state'].replace(state_map)


# 3. Create Composite Features for Matching

# Create a clean, combined address field
df_clean['full_address'] = (
    df_clean['street_number'] + ' ' + 
    df_clean['address_1'] + ' ' + 
    df_clean['address_2']
)
# Clean up extra spaces
df_clean['full_address'] = df_clean['full_address'].str.replace(r'\s+', ' ', regex=True).str.strip()

# 4. Final Check

print("\nCleaned Data Info")
df_clean.info()

print("\nCleaned Data Head")
print(df_clean.head())

print("\nCleaned State Values")
print(df_clean['state'].value_counts())


Cleaned Data Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     5000 non-null   object
 1   surname        5000 non-null   object
 2   street_number  5000 non-null   object
 3   address_1      5000 non-null   object
 4   address_2      5000 non-null   object
 5   suburb         5000 non-null   object
 6   postcode       5000 non-null   object
 7   state          5000 non-null   object
 8   date_of_birth  5000 non-null   object
 9   soc_sec_id     5000 non-null   int64 
 10  id             5000 non-null   int64 
 11  full_address   5000 non-null   object
dtypes: int64(2), object(10)
memory usage: 468.9+ KB

Cleaned Data Head
  given_name   surname street_number        address_1               address_2  \
0   mitchell     green             7    wallaby place                  delmar   
1     harley  mccarthy           177  

# Blocking

In [6]:
print(f"Number of duplicate 'id' values: {df_clean['id'].duplicated().sum()}")

Number of duplicate 'id' values: 127


In [7]:
import recordlinkage

# 1. Create a new indexer for blocking
indexer_blocked = recordlinkage.Index()

# 2. Define the blocking rule: only match records in the same 'postcode'
indexer_blocked.block('postcode')

# 3. Generate the candidate pairs using df_clean directly.
# It will use its default unique index (0-4999).
# The 'id' column remains as a normal data column.
candidate_pairs = indexer_blocked.index(df_clean)

# 4. Show the results
print("Blocking Results")

total_records = len(df_clean)
# Calculate the full pairs count manually
full_pairs_count = (total_records * (total_records - 1)) / 2

print(f"Total number of records: {total_records}")
print(f"Number of pairs WITHOUT blocking (calculated): {int(full_pairs_count)}")
print(f"Number of pairs WITH blocking on 'postcode': {len(candidate_pairs)}")

reduction = (1 - (len(candidate_pairs) / full_pairs_count)) * 100
print(f"Pairwise comparisons reduced by: {reduction:.2f}%")

Blocking Results
Total number of records: 5000
Number of pairs WITHOUT blocking (calculated): 12497500
Number of pairs WITH blocking on 'postcode': 16115
Pairwise comparisons reduced by: 99.87%


# Pairwise Comparison

In [8]:
import recordlinkage
import jellyfish # Make sure this is installed

# We need df_clean and candidate_pairs from the previous steps

print("Starting pairwise comparison for 16,115 pairs...")

# 1. Create the comparison object
compare_cl = recordlinkage.Compare()

# 2. Define the comparison rules
# String similarity (Jaro-Winkler) for noisy fields.
# Score 1.0 = perfect match, 0.0 = no match.
compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold=0.85, label='given_name_sim')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname_sim')
compare_cl.string('date_of_birth', 'date_of_birth', method='jarowinkler', threshold=0.9, label='dob_sim')
compare_cl.string('full_address', 'full_address', method='jarowinkler', threshold=0.85, label='address_sim')

# Exact match for the 'state' field we cleaned
compare_cl.exact('state', 'state', label='state_match')

# 3. Compute the similarities for all candidate pairs
# This will return a DataFrame with your 16,115 pairs
# and the 5 similarity scores for each.
feature_vectors = compare_cl.compute(candidate_pairs, df_clean)

# 4. Show the results
print("\n--- Comparison Feature Vectors (Head) ---")
print(feature_vectors.head())

print("\n--- Summary of Matches Found ---")
print(feature_vectors.sum(axis=0))

Starting pairwise comparison for 16,115 pairs...

--- Comparison Feature Vectors (Head) ---
       given_name_sim  surname_sim  dob_sim  address_sim  state_match
31 30             1.0          1.0      1.0          1.0            1
42 17             1.0          1.0      1.0          1.0            1
46 35             1.0          1.0      1.0          1.0            1
59 27             1.0          1.0      1.0          1.0            1
71 1              0.0          0.0      0.0          0.0            0

--- Summary of Matches Found ---
given_name_sim    3724.0
surname_sim       3983.0
dob_sim           4600.0
address_sim       4142.0
state_match       6737.0
dtype: float64


# Find Matches and Clustering

In [9]:
import networkx as nx

# feature_vectors is the DataFrame from your previous step

# Step 5: Decision Rule

# 1. Create a total 'match_score' by summing the 5 features
match_score = feature_vectors.sum(axis=1)

# 2. Let's look at the distribution of scores
# This helps us pick a good threshold.
print("Match Score Distribution")
print(match_score.value_counts().sort_index(ascending=False))

# 3. Set your threshold. A score of 4.0 or 5.0 is a very strong match.
# A score of 3.0 might also be a match. Let's start with 4.0
MATCH_THRESHOLD = 4.0 

# 4. Get all pairs that meet or exceed this threshold
matches = feature_vectors[match_score >= MATCH_THRESHOLD]
matching_pairs_list = matches.index.to_list()

print(f"\nFound {len(matching_pairs_list)} matching pairs (score >= {MATCH_THRESHOLD})")

# Step 6: Clustering

# 1. Create a graph from the list of matching pairs
G = nx.Graph()
G.add_edges_from(matching_pairs_list)

# 2. Find all connected components (the clusters)
clusters = list(nx.connected_components(G))

# 3. Show Final Results
print(f"Total number of clusters (people) found: {len(clusters)}")
print(f"Total number of records: {len(df_clean)}")

# Let's look at the first 5 clusters
print("\nSample Clusters (Row IDs)")
for i, cluster in enumerate(clusters[:5]):
    print(f"Cluster {i+1}: {cluster}")

Match Score Distribution
5.0    2216
4.0    1731
3.0     834
2.0     216
1.0    2248
0.0    8870
Name: count, dtype: int64

Found 3947 matching pairs (score >= 4.0)
Total number of clusters (people) found: 1059
Total number of records: 5000

Sample Clusters (Row IDs)
Cluster 1: {3872, 4333, 30, 31}
Cluster 2: {17, 42}
Cluster 3: {35, 46, 820, 2773, 600, 1210}
Cluster 4: {27, 59, 1535}
Cluster 5: {226, 138, 49, 2774, 2334}


# Evaluation

In [10]:
from sklearn.metrics import adjusted_rand_score
import pandas as pd

# 'clusters' is the list of clusters from your previous step
# 'df_clean' is your cleaned DataFrame

print("Evaluating Cluster Performance")

# 1. Create a "predicted labels" array
# First, create a mapping from row_id -> cluster_id
cluster_mapping = {}
for cluster_id, row_ids in enumerate(clusters):
    for row_id in row_ids:
        cluster_mapping[row_id] = cluster_id

# Now, create the 'predictions' array for all 5000 records
# We must ensure it's in the same order (0-4999) as df_clean
predictions = []
for row_id in range(len(df_clean)):
    # If a record was a singleton (not in any cluster),
    # give it a unique negative cluster_id
    predictions.append(cluster_mapping.get(row_id, -row_id - 1))

# 2. Get the "true labels" array
# This is just the 'soc_sec_id' column
truth = df_clean['soc_sec_id'].to_list()

# 3. Calculate and print the score
ars_score = adjusted_rand_score(truth, predictions)

print(f"\nGround Truth (soc_sec_id) Clusters: {df_clean['soc_sec_id'].nunique()}")
print(f"Algorithm (Predicted) Clusters: {len(clusters)}")
print(f"\nAdjusted Rand Score (ARS): {ars_score:.4f}")

if ars_score > 0.9:
    print("Result: Excellent! Your model's clusters are a near-perfect match.")
elif ars_score > 0.7:
    print("Result: Great! Your model captured most of the groups correctly.")
else:
    print("Result: Good. The model found a strong signal but could be tuned.")

Evaluating Cluster Performance

Ground Truth (soc_sec_id) Clusters: 2291
Algorithm (Predicted) Clusters: 1059

Adjusted Rand Score (ARS): 0.7307
Result: Great! Your model captured most of the groups correctly.


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:50: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(labels_pred)
